In [26]:
import os
import sys

from  IPython.core.debugger import Pdb 


sys.path.append("./exp/20220428/utils")

import argparse

from utils.load_data import (
    load_data,
    list2df,
    load_annotated_trackret_file,
    get_player_id_list,
    get_player_id_dict,
    load_bbox_data,
    load_pose_data,
)
from utils.visualize import (
    load_img,
    draw_labels,
    save_frame_img,
    show_image,
    get_masked_bboxes,
    get_masked_poses,
    tlwh2tldr,
    draw_id,
    draw_bbox,
    colors,
    draw_pose,
)
from utils.frame import sec2frame
import tqdm
from pdb import set_trace as pst
from ipdb import set_trace as ist
import pandas as pd
from pathlib import Path
from typing import Union, Tuple
import multiprocessing
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
def show_img_jupyter(im0):
    plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB)) # OpenCV は色がGBR順なのでRGB順に並べ替える
    plt.show()

In [3]:
video_name="p008"
dst_path=f"/home/mei/Documents/deep_sort/exp/20220512/output/{video_name}/img"
img_path=f"/home/mei/Documents/deep_sort/exp/20220512/data/{video_name}/img1"
tracklet_path=f"/home/mei/Documents/deep_sort/exp/20220512/data/{video_name}_annotated_tracklet.csv"
bbox_path=f"/home/mei/Documents/deep_sort/exp/20220512/data/{video_name}/{video_name}_deepsort_output.txt"

pose_path=f"/home/mei/Documents/deep_sort/exp/20220512/data/detection_poses_{video_name}.pkl"
max_frame=10000


MAX_FRAME: int = max_frame
img_path: Path = Path(img_path)
dst_path: Path = Path(dst_path)
bbox_path: Path = Path(bbox_path)
tracklet_path: Path = Path(tracklet_path)
pose_path: Path = Path(pose_path)

In [4]:
# tracklet data
tracklet_df = load_annotated_trackret_file(tracklet_path)
player1_tracklet, player2_tracklet = get_player_id_list(tracklet_df)

# pose data
pose_data = load_pose_data(pose_path)

bbox_list: list = load_bbox_data(str(bbox_path))

In [5]:
tracklet_df.head()

,frame,player1,player2
0,3,3,2
1,15,3,2
2,114,6,2
3,190,6,2
4,232,10,2


In [6]:
bbox_list[:1]

[[3, 1, 1674.26, 386.02, 243.47, 445.98, 1, -1, -1, -1]]

In [7]:
pose_data[:1]

[[1,
  0,
  array([[1.85680774e+03, 4.58693726e+02, 7.82352149e-01],
         [1.86800659e+03, 4.47096222e+02, 0.00000000e+00],
         [1.86035339e+03, 4.48209869e+02, 0.00000000e+00],
         [1.88326233e+03, 4.39125458e+02, 7.15171933e-01],
         [1.87934204e+03, 4.43409607e+02, 0.00000000e+00],
         [1.92140149e+03, 4.91135284e+02, 0.00000000e+00],
         [1.89605811e+03, 4.86776459e+02, 0.00000000e+00],
         [1.90395435e+03, 5.85826660e+02, 0.00000000e+00],
         [1.86885608e+03, 5.79733643e+02, 0.00000000e+00],
         [1.87056934e+03, 5.12566223e+02, 5.87612212e-01],
         [1.83420227e+03, 6.05726196e+02, 5.86196184e-01],
         [1.89888025e+03, 6.49293091e+02, 0.00000000e+00],
         [1.87957764e+03, 6.34314575e+02, 0.00000000e+00],
         [1.74900854e+03, 6.52701355e+02, 0.00000000e+00],
         [1.74703516e+03, 6.45243286e+02, 5.72371662e-01],
         [1.75957446e+03, 7.90399170e+02, 0.00000000e+00],
         [1.76161206e+03, 7.72725403e+02, 0.00

In [8]:
target_frame_id = 3
im0 = load_img(img_path,target_frame_id)
target_bboxes: list = get_masked_bboxes(bbox_list, target_frame_id)
target_poses: list = get_masked_poses(pose_data, target_frame_id)

In [9]:
display(target_bboxes)
display(target_poses)

[[3, 1, 1674.26, 386.02, 243.47, 445.98, 1, -1, -1, -1],
 [3, 2, 1262.48, 259.78, 240.23, 448.46, 1, -1, -1, -1],
 [3, 3, 15.75, 249.19, 205.6, 740.83, 1, -1, -1, -1],
 [3, 4, 1585.67, 242.0, 82.24, 178.46, 1, -1, -1, -1]]

[[3,
  0,
  array([[1.35130103e+03, 3.04378815e+02, 5.55532336e-01],
         [1.36033154e+03, 2.97190338e+02, 7.08667874e-01],
         [1.34987622e+03, 2.97692200e+02, 5.24191260e-01],
         [1.38551562e+03, 3.02605591e+02, 9.04767036e-01],
         [1.34110156e+03, 3.05970398e+02, 0.00000000e+00],
         [1.41575537e+03, 3.51282623e+02, 7.67476022e-01],
         [1.33234888e+03, 3.60924713e+02, 8.55389237e-01],
         [1.44599878e+03, 4.04571625e+02, 8.64046216e-01],
         [1.31584802e+03, 4.12362122e+02, 8.29691470e-01],
         [1.47220898e+03, 4.61305328e+02, 9.02355790e-01],
         [1.28655859e+03, 4.34210785e+02, 8.23358238e-01],
         [1.40576270e+03, 4.81938232e+02, 0.00000000e+00],
         [1.35049438e+03, 4.80507202e+02, 5.94360292e-01],
         [1.43809387e+03, 5.74655029e+02, 8.25778067e-01],
         [1.31692822e+03, 5.68984497e+02, 8.65403473e-01],
         [1.47158008e+03, 6.70040283e+02, 8.37401152e-01],
         [1.31933374e+03, 6.56868530e+02, 6.88

In [22]:
def get_pose_bbox_position(pose):
    """姿勢のbbox領域を獲得する
    Args:
        pose list(float): キーポイントの座標
    Returns:
        top_left, down_right: 左上(x,y), 右上(x,y)
    """
    INF = 1<<20
    min_x, max_x = INF, -1
    min_y, max_y = INF, -1
    
    for x,y,_ in pose:
        x,y = float(x),float(y)
        min_x = min(min_x,x)
        max_x = max(max_x,x)
        min_y = min(min_y,y)
        max_y = max(max_y,y)
    
    left_top = (min_x,min_y)
    right_down = (max_x,max_y)
    
    return left_top, right_down

def calc_IOU_score(a, b):
    """IOUを計算する
    Args:
        a: righttop, leftdown
        b: righttop, leftdown
    Returns:
        IoU score: float
    """

    (min_ax, min_ay), (max_ax, max_ay) = a[0], a[1]
    (min_bx, min_by), (max_bx, max_by) = b[0], b[1]
    
    a_area = (max_ax - min_ax + 1) * (max_ay - min_ay + 1)
    b_area = (max_bx - min_bx + 1) * (max_by - min_by + 1)
    
    inner_min_x = max(min_ax,min_bx)
    inner_max_x = min(max_ax,max_bx)
    inner_min_y = max(min_ay,min_by)
    inner_max_y = min(max_ay,max_by)
    
    w = max(0, inner_max_x - inner_min_x + 1)
    h = max(0, inner_max_y - inner_min_y + 1)
    intersect = w*h

    iou = intersect / (a_area + b_area - intersect)
    
    return iou

In [23]:
target_bboxes

[[3, 1, 1674.26, 386.02, 243.47, 445.98, 1, -1, -1, -1],
 [3, 2, 1262.48, 259.78, 240.23, 448.46, 1, -1, -1, -1],
 [3, 3, 15.75, 249.19, 205.6, 740.83, 1, -1, -1, -1],
 [3, 4, 1585.67, 242.0, 82.24, 178.46, 1, -1, -1, -1]]

In [31]:
for pose in target_poses:
    _,_, pose = pose
    pose_topleft, pose_rightdown = get_pose_bbox_position(pose)
    
    scores: list = []
    for bbox in target_bboxes:
        bbox = bbox[2:6]
        bbox = tlwh2tldr(bbox)
        bbox_topleft, bbox_rightdown = (bbox[0],bbox[1]),(bbox[2],bbox[3])
        score = calc_IOU_score((pose_topleft,pose_rightdown),(bbox_topleft,bbox_rightdown))
        scores.append(score)
    
    scores = np.array(scores)
    max_idx = np.argmax(scores)
    
    matched_tracklet = target_poses[max_idx][1]
    print(matched_tracklet)


1
0
2
3
0
0
0
